In [1]:
library(jsonlite)
library(rlist)
library(data.table)
suppressPackageStartupMessages(library("dplyr"))
library(XML)
library(methods)

In [2]:
# We set the working directory
setwd('/home/jovyan/xml/')


In [ ]:
# We select the xmls with the datas we need
suppressWarnings({
options(stringsAsFactors = FALSE)
xmlproducts3=list.files(pattern='product3')
# We put all the xml products3 from orphadata in data frames
indice_list=1
xmldoc=list()
products3=list()
node_names=list()
names=list()
for (i in 1:length(xmlproducts3)) 
{
  xmldoc[[indice_list]]=xmlParse(xmlproducts3[i]) 
  products3[[indice_list]]=xmlToList(xmldoc[[indice_list]])
  indice_list=indice_list+1
}
})

In [ ]:
# We get the Classification information
prods3=list()
for (i in 1:length(products3))
{
  prods3[[i]]=unlist(products3[[i]]$ClassificationList)
}
# Each element in the list is a disorder

In [ ]:
# We get the OrphaCode and the DisorderType for each disorder
disordertype = list()
orpha = list()
indice_orpha = 1
indice_disordertype = 1
for (i in 1:length(prods3))
  for (j in 1:length(prods3[[i]])) 
    if (grepl("Disorder.OrphaCode", names(prods3[[i]][j]))) {
      orpha[[indice_orpha]] = data.frame(classification = prods3[[i]][2],
                                      orpha = prods3[[i]][j])
      indice_orpha = indice_orpha + 1
    } else if (grepl("DisorderType..attrs.id", names(prods3[[i]][j]))) {
      disordertype[[indice_disordertype]] = data.frame(classification = prods3[[i]][2],
                                              disordertype = prods3[[i]][j])
      indice_disordertype = indice_disordertype + 1
    } 

disordertype = rbindlist(disordertype)
orpha = rbindlist(orpha)

In [ ]:
# We only keep Disorders
disorders = cbind.data.frame(orpha, disordertype = disordertype$disordertype)
disorders = filter(disorders, disordertype == "21429" |
              disordertype == "21401" |
              disordertype == "21415" |
              disordertype == "21394" |
              disordertype == "21422" |
              disordertype == "21408")

disorders = filter(disorders, !duplicated(disorders))

In [ ]:
# We count the number of OrphaCodes in each xml (in each medical speciality)
medical_speciality = data.frame(table(disorders$classification))
names(medical_speciality)=c('Medical Speciality','Number')
medical_speciality